<a href="https://colab.research.google.com/github/Aayushmatkar/DataScienceApplications/blob/main/Final_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
#MODEL IMPORTS
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

#VALIDATION METRICS
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

#visualisation
import matplotlib.pyplot as pl

In [38]:
%pip install mlxtend --upgrade

import math
import nltk
import pandas as pd 
import numpy as np
import re
import matplotlib.pyplot as plt
from nltk.corpus import gutenberg
from sklearn.utils import shuffle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, plot_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from mlxtend.evaluate import bias_variance_decomp
from sklearn import preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.2 MB/s eta 0:00:00
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [39]:
import nltk
from nltk.corpus import gutenberg
import re
import os,random
import pandas as pd
nltk.download('punkt')
nltk.download('gutenberg')

def create_partitions(text, book_name, author):
    partitions = []
    words = nltk.word_tokenize(text)
    for i in range(0, len(words), 100):
        partitions.append((words[i:i+100], book_name, author))
    return partitions

#The books selected below are from the gutenberg module and are of the same genre 'novel'
book_names=['melville-moby_dick.txt','edgeworth-parents.txt','chesterton-thursday.txt','austen-emma.txt','shakespeare-hamlet.txt']

def random_sentences_with_book_name():
  result=pd.DataFrame(columns=['Text', 'Book Name','Author'])
  for i in book_names:
    # Splitting the book names to remove their 'txt' extension
    label_name=i.split('.')[0]

    # Getting the book names from Gutenberg and reading them
    book = gutenberg.raw(i)

    #Getting the book names and their respective authors
    lines = book.splitlines()

    #The first line consists information about the above comment
    line1=lines[0]
    
    #Removing the year from the line
    pattern = r'\D+'

    digits_removed = re.findall(pattern, line1)

    #Getting the author name
    author = re.search(r'\sby\s([\w\s\.]+)[\s\d{4}]+', digits_removed[0]).group(1)

    #Getting the book name
    book_name = re.search(r'\[([\w\s\']+)[\,]?\sby', line1).group(1)

    # Create partitions of 100 words each
    partitions = create_partitions(book, book_name, author)

    # Create 200 random samples of the partitions
    samples = random.sample(partitions, 200)

    # Using regular expressions to clean the data
    for i in range(len(samples)):
        samples[i] = (re.sub(r'[^\w\s]','', ' '.join(samples[i][0])), samples[i][1], samples[i][2])

    # Serialize the data using Pandas
    df = pd.DataFrame(samples, columns=['Text', 'Book Name','Author'])

    #Appending the random sentences of a book to the final dataframe 
    result=pd.concat([result,df])
  result.to_csv('book_partitions.csv', index=False)

  print("Data saved to book_partitions.csv.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [40]:
random_sentences_with_book_name()

Data saved to book_partitions.csv.


In [41]:
data=pd.read_csv('book_partitions.csv')

In [42]:
data['Author'].value_counts(),data['Author'].unique()

(Herman Melville        200
 Maria                  200
 G. K. Chesterton       200
 Jane Austen            200
 William Shakespeare    200
 Name: Author, dtype: int64,
 array(['Herman Melville', 'Maria', 'G. K. Chesterton', 'Jane Austen',
        'William Shakespeare'], dtype=object))

In [43]:
data['Book Name'].value_counts(),data['Book Name'].unique()

(Moby Dick                   200
 The Parent's Assistant      200
 The Man Who Was Thursday    200
 Emma                        200
 The Tragedie of Hamlet      200
 Name: Book Name, dtype: int64,
 array(['Moby Dick', "The Parent's Assistant", 'The Man Who Was Thursday',
        'Emma', 'The Tragedie of Hamlet'], dtype=object))

In [57]:
data['label']='a'
data['index']=0

In [58]:
data['label'][200:400]='b'
data['index'][200:400]=1
data['label'][400:600]='c'
data['index'][400:600]=2
data['label'][600:800]='d'
data['index'][600:800]=3
data['label'][800:1000]='e'
data['index'][800:1000]=4


<ipython-input-58-3b20635868f0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'][200:400]='b'
<ipython-input-58-3b20635868f0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['index'][200:400]=1
<ipython-input-58-3b20635868f0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'][400:600]='c'
<ipython-input-58-3b20635868f0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [59]:
data

,Text,Book Name,Author,label,index
612,much of both that he is glad to get rid of th...,Emma,Jane Austen,a,0
263,you wo nt make me an example for the rest It ...,The Parent's Assistant,Maria,a,0
569,spilt and shining you would still be standing...,The Man Who Was Thursday,G. K. Chesterton,a,0
673,and _he_ would never own that he had Concessi...,Emma,Jane Austen,a,0
79,where it is furnished with a movable sidescree...,Moby Dick,Herman Melville,a,0
...,...,...,...,...,...
575,said Dr Bull in adamantine humility it is I ...,The Man Who Was Thursday,G. K. Chesterton,e,4
297,had given him leave to have a boy in the house...,The Parent's Assistant,Maria,e,4
599,care that it was not supreme I tell you he ha...,The Man Who Was Thursday,G. K. Chesterton,e,4
176,is it not hard that with this weary load I be...,Moby Dick,Herman Melville,e,4


In [60]:
data = shuffle(data)
data.head()

X_data = data['Text']
Y_data = data['label']

print(X_data.shape)
print(Y_data.shape)

# Split data to train and test
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_data, Y_data, test_size= 0.3, random_state= 42)

(1000,)
(1000,)


In [61]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0,5)):
  
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, '-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, '-', color="g",
             label="Cross-validation score")

    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left');
    return plt

In [62]:
def which_one_misclassify(X_train, y_train, X_test, y_test ,y_pred):
  errors  = [] 
  correct = []
  predict = []
  gg=np.array(y_test)

  for index ,text in enumerate(X_test_df) :
    if y_pred[index] != gg[index]:
      error = text
      errors.append(error)
      corr = gg[index]
      correct.append(corr)
      pred = y_pred[index]
      predict.append(pred)
  document_df = pd.DataFrame()
  document_df['doc_error'] = errors
  document_df['correct']   = correct
  document_df['Predicted'] = predict
  #document_df['book_name'] = 
  print(" The Documents that the Model Misclassify are  : " , len(errors))
  print("----------------------------------------------------------------------------------------------\n")

  #label_encoder object knows how to understand word labels. 
  label_encoder = preprocessing.LabelEncoder()
  X_train_copy = np.copy(X_train)
  X_test_copy = np.copy(X_test)
  y_train_copy = np.copy(y_train)
  y_test_copy = np.copy(y_test)

  avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
         general_model, np.array(X_train_copy)  , label_encoder.fit_transform(y_train_copy)  , np.array(X_test_copy)  , label_encoder.fit_transform(y_test_copy),num_rounds=2, random_seed=123)
  
  print('Average bias: %.3f' % avg_bias)
  print('Average variance: %.3f' % avg_var)
  print("----------------------------------------------------------------------------------------------")

  return document_df 

In [63]:
def build_model(model, model_name, X_train, X_test, y_train, y_test, cv ):

  global general_model
  general_model = model.fit(X_train, y_train) # train the model
  # make prediction of X_test and print Classification report
  global y_pred
  y_pred = model.predict(X_test)
  # 2- Applying K_Fold Cross Vaildation

  cv_accuracies = cross_val_score(estimator = model , X= X_train , y = y_train ,cv = cv)
  accuracy_avg = cv_accuracies.mean()# measure the accuracy of the model (bais)
  test_accuracy = accuracy_score(y_test, y_pred)
  # Evaluate the model performance using metrics.accuracy_score to measure the score
  print("cross_validation accuracy :  ", cv_accuracies)
  print("\n average cross validation accuracy :  " , accuracy_avg)
  print("\n testing accuracy :  "  ,test_accuracy,"\n")
  print("----------------------------------------------------------------------------------------------")

 

  print(classification_report(y_test, y_pred))

In [64]:
count_Vec= CountVectorizer()
bow = count_Vec.fit_transform(data['Text'])
bow_df = pd.DataFrame(bow.toarray(), columns=count_Vec.get_feature_names())
bow_df

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,10800,11,114,125,1599,1652,1671,1729,1791,1851,...,yourselves,youth,youthful,zare,ze,zealand,zem,zmite,zodiac,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
X = bow.toarray()
Y = data['label']

print(X.shape)
print(Y.shape)

# Split data to train and test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size= 0.3, random_state= 42)

(1000, 10295)
(1000,)


In [66]:
decision_tree_BOW = DecisionTreeClassifier(random_state=0, max_depth= 10, criterion= 'entropy', min_samples_leaf= 1)

build_model(decision_tree_BOW,"Decsion Tree With BOW", X_train, X_test, y_train, y_test ,10)

DT_Bow_error_df = which_one_misclassify(X_train, y_train, X_test, y_test ,y_pred)
display(DT_Bow_error_df.head())

cross_validation accuracy :   [0.21428571 0.22857143 0.11428571 0.18571429 0.2        0.11428571
 0.15714286 0.21428571 0.2        0.24285714]

 average cross validation accuracy :   0.18714285714285714

 testing accuracy :   0.17 

----------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

           a       0.21      0.07      0.10        61
           b       0.16      0.05      0.07        63
           c       0.21      0.22      0.21        73
           d       0.15      0.55      0.23        49
           e       0.33      0.02      0.04        54

    accuracy                           0.17       300
   macro avg       0.21      0.18      0.13       300
weighted avg       0.21      0.17      0.13       300

 The Documents that the Model Misclassify are  :  249
----------------------------------------------------------------------------------------------



TypeError: ignored

In [54]:
knn_bow = KNeighborsClassifier(n_neighbors = 3, algorithm= 'kd_tree', p= 1)
build_model(knn_bow,"KNN with BOW", X_train, X_test, y_train, y_test ,10)

KNN_Bow_error_df = which_one_misclassify(X_train, y_train, X_test, y_test ,y_pred)
display(KNN_Bow_error_df.head())

cross_validation accuracy :   [0.57142857 0.54285714 0.57142857 0.48571429 0.54285714 0.67142857
 0.58571429 0.64285714 0.72857143 0.57142857]

 average cross validation accuracy :   0.5914285714285714

 testing accuracy :   0.56 

----------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

           a       0.53      0.55      0.54        64
           b       0.35      0.64      0.45        56
           c       0.67      0.59      0.63        64
           d       0.69      0.57      0.62        60
           e       1.00      0.45      0.62        56

    accuracy                           0.56       300
   macro avg       0.65      0.56      0.57       300
weighted avg       0.65      0.56      0.57       300

 The Documents that the Model Misclassify are  :  132
----------------------------------------------------------------------------------------------



TypeError: ignored

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
#MODEL IMPORTS
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

#VALIDATION METRICS
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

#visualisation
import matplotlib.pyplot as plt

In [2]:
data = pd.DataFrame()
X_data = data['100_Words']
Y_data = data['label']

print(X_data.shape)
print(Y_data.shape)

# Split data to train and test
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_data, Y_data, test_size= 0.3, random_state= 42)

NameError: ignored